In [34]:
# Standard imports
import time

# Databass imports
from databass import *

simple_test = [
    "SELECT * FROM data",
    "SELECT a, b FROM data",
    "SELECT data.e FROM data GROUP BY data.e",
    "SELECT data.a, data4.a FROM data, data4 WHERE data.a = data4.a"
]

experiment_one = [
    "SELECT * from lineorder LIMIT 1",
    "SELECT lo_custkey, lo_suppkey FROM lineorder, supplier WHERE lineorder.lo_suppkey = supplier.s_suppkey",
    "SELECT sum(lo_extendedprice * lo_discount) AS revenue FROM lineorder, date WHERE lo_orderdate = d_datekey AND d_year = 1993 AND lo_discount BETWEEN 1 AND 3 AND lo_quantity < 25"
]

experiment_two = [
    "SELECT lo_orderkey FROM lineorder LIMIT 20",
    "SELECT p_category FROM part",
    "SELECT c_nation FROM customer WHERE c_nation = 'UNITED STATES'"
]

def run_query(db, qstr):
    plan = parse(qstr)
    plan = plan.to_plan()
    print("QUERY PLAN", plan.pretty_print())
    return run_plan(db, plan)

def run_plan(db, plan):
    databass_rows = list()
    plan = Optimizer(db)(plan)
    # plan = Optimizer(db)(plan)
    for row in plan:
        vals = []
        for v in row:
            if isinstance(v, str):
                vals.append(v)
            else:
                vals.append(float(v))
        databass_rows.append(vals)
    return databass_rows

def setup_row():
    print("=== ROW MODE: SETUP ===\n")
    mode = Mode.ROW # either Mode.ROW or Mode.COLUMN_ALL or Mode.COLUMN_SELECT
    print("[setup] db in mode...", mode)
    db = Database.db(mode)
    print("[setup] {num} tables...OK".format(num=len(db.tablenames)), db.tablenames)
    print("[setup] ...OK")

    print("\n=== ROW MODE: RUNNING QUERIES ===\n")

    for qstr in simple_test:
        print("[query]", qstr)
        start = time.time()
        output = run_query(db, qstr)
        # print("[output] ", output)
        print("[query] took %0.5f sec\n" % (time.time()-start))

def setup_col():
    print("=== RUNNING IN COL MODE ===\n")
    mode = Mode.COLUMN_ALL # either Mode.ROW or Mode.COLUMN_ALL or Mode.COLUMN_SELECT
    print("[setup] db in mode...", mode)
    db = Database.db(mode)
    print("[setup] {num} tables...OK".format(num=len(db.tablenames)), db.tablenames)
    print("[setup] ...OK")
    return db

In [2]:
db = setup_col()

=== RUNNING IN COL MODE ===

[setup] db in mode... Mode.COLUMN_ALL
[table] customer with schema: c_custkey:num, c_name:str, c_address:str, c_city:str, c_nation:str, c_region:str, c_phone:str, c_mktsegment:str
[table] data with schema: a:num, b:num, c:num, d:num, e:str, f:num, g:str
[table] data2 with schema: a:num, b:num, c:num, d:num, e:str, f:num, g:str
[table] data3 with schema: a:num, b:num
[table] data4 with schema: a:num, b:num, c:num, d:num, e:str, f:num, g:str
[table] data_orig with schema: a:num, b:num, c:num, d:num
[table] date with schema: d_datekey:num, d_date:str, d_dayofweek:str, d_month:str, d_year:num, d_yearmonthnum:num, d_yearmonth:str, d_daynuminweek:num, d_daynuminmonth:num, d_daynuminyear:num, d_monthnuminyear:num, d_weeknuminyear:num, d_sellingseason:str, d_lastdayinweekfl:num, d_lastdayinmonthfl:num, d_holidayfl:num, d_weekdayfl:num
[table] lineorder with schema: lo_orderkey:num, lo_linenumber:num, lo_custkey:num, lo_partkey:num, lo_suppkey:num, lo_orderdate:num,

In [35]:
def run_exp(db, queries):
    print("\n=== RUNNING QUERIES ===\n")

    for qstr in queries:
        print("[query] ", qstr)
        start = time.time()
        output = run_query(db, qstr)
        print("[query] took %0.5f sec\n" % (time.time()-start))
        print("[output] ", output)
    
    print("\n=== DONE WITH ALL QUERIES ===\n")

In [36]:
run_exp(db, experiment_two[0:1])


=== RUNNING QUERIES ===

[query]  SELECT lo_orderkey FROM lineorder LIMIT 20
QUERY PLAN 
Project(lineorder.lo_orderkey:num AS lo_orderkey)
  LIMIT(20.0 OFFSET 0)
    From()
      Scan(lineorder AS lineorder)
scan schema lineorder.lo_orderkey:num, lineorder.lo_linenumber:num, lineorder.lo_custkey:num, lineorder.lo_partkey:num, lineorder.lo_suppkey:num, lineorder.lo_orderdate:num, lineorder.lo_orderpriority:str, lineorder.lo_shippriority:num, lineorder.lo_quantity:num, lineorder.lo_extendedprice:num, lineorder.lo_ordtotalprice:num, lineorder.lo_discount:num, lineorder.lo_revenue:num, lineorder.lo_supplycost:num, lineorder.lo_tax:num, lineorder.lo_commitdate:num, lineorder.lo_shipmode:str
Scan(lineorder AS lineorder)
From()
LIMIT(20.0 OFFSET 0)
Project(lineorder.lo_orderkey:num AS lo_orderkey)
scan schema lineorder.lo_orderkey:num, lineorder.lo_linenumber:num, lineorder.lo_custkey:num, lineorder.lo_partkey:num, lineorder.lo_suppkey:num, lineorder.lo_orderdate:num, lineorder.lo_orderprior